In [1]:
library(rhdf5)
library(Matrix)
library(SingleCellExperiment)

Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:Matrix’:

    which

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, 

In [2]:
filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/all_da_d52_cells.h5"

In [3]:
cbind(h5ls(filename)$group,
h5ls(filename)$name)

/,X
/X,data
/X,indices
/X,indptr
/,obs
/obs,__categories
/obs/__categories,batch
/obs/__categories,celltype
/obs/__categories,donor_id
/obs/__categories,pool_id
/obs/__categories,sample_id


In [4]:
ind = h5read(file = filename, "obs")$index # cell names
var = h5read(file = filename, "var") # gene names

In [6]:
head(ind)

[1] "CCAGCGAGTTACTGAC-1-64" "ACGCAGCGTTCGCTAA-1-45" "ACGTCAAGTCTAGCCG-1-51"
[4] "GACACGCGTGTGAAAT-1-19" "CGTCTACGTCGGGTCT-1-35" "AAATGCCTCGGATGGA-1-53"

In [7]:
head(var$index)
# var$index[grep("NANOG",var$index)]

[1] "MIR1302-10"   "FAM138A"      "OR4F5"        "RP11-34P13.7" "RP11-34P13.8"
[6] "AL627309.1"

In [8]:
X = h5read(file = filename, "X")
X_data = as.integer(X$data)
X_indptr = as.integer(X$indptr)
X_indices = as.integer(X$indices)
rm(X)

In [9]:
sMat = sparseMatrix(
i = X_indices,
p = X_indptr,
x = X_data,
dims = c(length(var[[1]]), length(ind)),
index1 = FALSE
)
rownames(sMat) <- as.character(var[[1]])
colnames(sMat) <- as.character(ind)

In [10]:
rownames(sMat)[grep("SOX",rownames(sMat))]

[1] "QSOX1"     "SOX13"     "SOX11"     "SOX14"     "SOX2"      "SOX30"    
 [7] "SOX4"      "SOX3"      "SOX7"      "SOX7-1"    "SOX17"     "QSOX2"    
[13] "SOX6"      "SOX5"      "SOX21"     "SOX21-AS1" "SOX1"      "SOX8"     
[19] "SOX15"     "SOX9"      "SOX12"     "SOX18"     "SOX10"

In [11]:
R_filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/all_da_d52_cells.Rds"
saveRDS(sMat, file = R_filename)

In [22]:
obs = h5read(file = filename, "obs")
all_cols = names(obs)
all_cols
categorical_cols = names(h5read(file = filename, "obs/__categories"))
categorical_cols

[1] "__categories"                "batch"                      
 [3] "celltype"                    "cluster_id"                 
 [5] "donor_id"                    "index"                      
 [7] "log1p_n_genes_by_counts"     "log1p_total_counts"         
 [9] "log1p_total_counts_mito"     "n_counts_all"               
[11] "n_genes_by_counts"           "pct_counts_in_top_100_genes"
[13] "pct_counts_in_top_200_genes" "pct_counts_in_top_500_genes"
[15] "pct_counts_in_top_50_genes"  "pct_counts_mito"            
[17] "pool_id"                     "sample_id"                  
[19] "time_point"                  "total_counts"               
[21] "total_counts_mito"           "treatment"

[1] "batch"      "celltype"   "donor_id"   "pool_id"    "sample_id" 
[6] "time_point" "treatment"

In [54]:
df = data.frame(row.names = obs$index)
head(df)

CCAGCGAGTTACTGAC-1-64
ACGCAGCGTTCGCTAA-1-45
ACGTCAAGTCTAGCCG-1-51
GACACGCGTGTGAAAT-1-19
CGTCTACGTCGGGTCT-1-35
AAATGCCTCGGATGGA-1-53


In [55]:
for (col in all_cols){
#     print(col)
    if (!(col %in% c(categorical_cols, "__categories","index"))){
#         print(col)
#         print(obs[[col]])
        df[[col]] = (as.numeric(unlist(obs[[col]])))   
    }
    if (col %in% c(categorical_cols)){
        colname = paste0("obs/__categories/",col)
        df[[col]] = h5read(file = filename,colname)[((as.numeric(obs[[col]]))+1)]
    }
}

In [56]:
head(df,2)

,batch,celltype,cluster_id,donor_id,log1p_n_genes_by_counts,log1p_total_counts,log1p_total_counts_mito,n_counts_all,n_genes_by_counts,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,pct_counts_in_top_50_genes,pct_counts_mito,pool_id,sample_id,time_point,total_counts,total_counts_mito,treatment
CCAGCGAGTTACTGAC-1-64,64,DA,3,HPSI0513i-veve_2,8.417373,9.667068,6.216606,15788,4524,28.88903,38.44059,54.28807,21.56068,3.166962,pool11,5245STDY7923746,D52,15788,500,NONE
ACGCAGCGTTCGCTAA-1-45,45,DA,3,HPSI1013i-hiaf_1,7.995307,9.307104,5.327876,11015,2966,39.48252,51.52973,67.32637,28.35225,1.861099,pool6,5245STDY7685556,D52,11015,205,ROT
ACGTCAAGTCTAGCCG-1-51,51,DA,3,HPSI1014i-babz_3,7.956126,8.983565,5.327876,7970,2852,34.84316,46.18570,61.96989,25.10665,2.572145,pool8,5245STDY7685562,D52,7970,205,NONE
GACACGCGTGTGAAAT-1-19,19,DA,3,HPSI1213i-hehd_1,7.880804,8.988196,4.744932,8007,2645,37.29237,48.47009,65.04309,27.77570,1.423754,pool4,5245STDY7544161,D52,8007,114,ROT
CGTCTACGTCGGGTCT-1-35,35,DA,3,HPSI0215i-hipn_1,8.022569,9.096051,5.902633,8919,3048,30.46306,41.58538,58.84068,21.61677,4.092387,pool7,5245STDY7660072,D52,8919,365,NONE
AAATGCCTCGGATGGA-1-53,53,DA,3,HPSI0715i-aowh_2,7.765145,8.867146,5.420535,7094,2356,42.09191,53.12941,68.70595,31.26586,3.171694,pool8,5245STDY7685564,D52,7094,225,ROT


In [57]:
cData_raw = df
cData_raw$index = rownames(cData_raw)
head(cData_raw,2)

,batch,celltype,cluster_id,donor_id,log1p_n_genes_by_counts,log1p_total_counts,log1p_total_counts_mito,n_counts_all,n_genes_by_counts,pct_counts_in_top_100_genes,...,pct_counts_in_top_500_genes,pct_counts_in_top_50_genes,pct_counts_mito,pool_id,sample_id,time_point,total_counts,total_counts_mito,treatment,index
CCAGCGAGTTACTGAC-1-64,64,DA,3,HPSI0513i-veve_2,8.417373,9.667068,6.216606,15788,4524,28.88903,...,54.28807,21.56068,3.166962,pool11,5245STDY7923746,D52,15788,500,NONE,CCAGCGAGTTACTGAC-1-64
ACGCAGCGTTCGCTAA-1-45,45,DA,3,HPSI1013i-hiaf_1,7.995307,9.307104,5.327876,11015,2966,39.48252,...,67.32637,28.35225,1.861099,pool6,5245STDY7685556,D52,11015,205,ROT,ACGCAGCGTTCGCTAA-1-45


In [58]:
cData <- as.data.frame(cData_raw)
cData_R_filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/all_da_d52_cells_cData.Rds"
saveRDS(cData, file = cData_R_filename)

In [59]:
# build the sce object
sce <- SingleCellExperiment(
assays = list(counts = sMat),
colData = cData)
sce

class: SingleCellExperiment 
dim: 32738 79801 
metadata(0):
assays(1): counts
rownames(32738): MIR1302-10 FAM138A ... AC002321.2 AC002321.1
rowData names(0):
colnames(79801): CCAGCGAGTTACTGAC-1-64 ACGCAGCGTTCGCTAA-1-45 ...
  CTTAGGAAGATTACCC-1-13 TCAACGACATAGAAAC-1-39
colData names(21): batch celltype ... treatment index
reducedDimNames(0):
spikeNames(0):
altExpNames(0):

In [60]:
sce_R_filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/all_da_d52_cells_sce.Rds"
saveRDS(sce, file = sce_R_filename)